# InsightFace-Powered Face Recognition System (GPU/CPU Compatible)
# Author: Code GPT 🧠

"""
Fully upgraded notebook using InsightFace — compatible with WSL2 (Ubuntu) and Conda.
Optimized for GPU if available (onnxruntime-gpu), automatically falls back to CPU.
Supports multiple reference faces and performs real-time recognition from webcam or video.
"""


1. Environment Setup Notes
* In your Conda environment, before running this notebook:

* conda create -n face_env python=3.9 -y
* conda activate face_env
* conda install -c conda-forge opencv numpy matplotlib -y
* pip install insightface onnxruntime-gpu faiss-cpu tqdm
* (If GPU drivers not configured, it will fall back automatically to onnxruntime CPU)

'''

In [9]:
import cv2
import numpy as np
import insightface
from tqdm import tqdm
import os
from pathlib import Path
from numpy.linalg import norm
from IPython.display import clear_output, Image, display

In [10]:
def safe_insightface_init():
    try:
        app = insightface.app.FaceAnalysis(name='buffalo_l',
                                           providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
        app.prepare(ctx_id=0, det_size=(640, 640))
        print("✅ InsightFace initialized with GPU if available.")
    except Exception as e:
        print(f"⚠️ GPU unavailable or CUDA issue: {e}\nSwitching to CPU mode.")
        app = insightface.app.FaceAnalysis(name='buffalo_l', providers=['CPUExecutionProvider'])
        app.prepare(ctx_id=0, det_size=(640, 640))
        print("✅ Running smoothly on CPU.")
    return app

app = safe_insightface_init()


2026-01-07 06:15:47.368522715 [E:onnxruntime:Default, provider_bridge_ort.cc:1992 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1637 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.12: cannot open shared object file: No such file or directory

2026-01-07 06:15:47.915430310 [E:onnxruntime:Default, provider_bridge_ort.cc:1992 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1637 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.12: cannot open shared object file: No such file or directory

2026-01-07 06:15:47.944739130 [E:onnxruntime:Default, provider_bridge_ort.cc:1992 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu_usr/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu_usr/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu_usr/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu_usr/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0


2026-01-07 06:15:48.127800624 [E:onnxruntime:Default, provider_bridge_ort.cc:1992 TryGetProviderInfo_CUDA] /onnxruntime_src/onnxruntime/core/session/provider_bridge_ort.cc:1637 onnxruntime::Provider& onnxruntime::ProviderLibrary::Get() [ONNXRuntimeError] : 1 : FAIL : Failed to load library libonnxruntime_providers_cuda.so with error: libcublasLt.so.12: cannot open shared object file: No such file or directory



Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu_usr/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
✅ InsightFace initialized with GPU if available.


In [11]:
# =============================
# 2. Load Reference Faces
# =============================
reference_folder = Path('data_check')
if not reference_folder.exists():
    raise FileNotFoundError("❌ Folder 'data_check' not found. Please create it and add known person images.")

known_faces = []
known_names = []

for image_path in reference_folder.glob('*.jpg'):
    img = cv2.imread(str(image_path))
    if img is None:
        print(f"⚠️ Unable to read image: {image_path.name}")
        continue

    faces = app.get(img)
    if len(faces) == 0:
        print(f"⚠️ No face detected in {image_path.name}")
        continue

    known_faces.append(faces[0].embedding)
    known_names.append(image_path.stem)

if len(known_faces) == 0:
    raise Exception("No valid reference faces found.")

known_faces = np.array(known_faces)
print(f"✅ Loaded {len(known_faces)} reference faces: {known_names}")


✅ Loaded 1 reference faces: ['chris_patt_img_test']


In [12]:
# =============================
# 3. Matching Utility
# =============================
def cosine_similarity(a, b):
    return np.dot(a, b) / (norm(a) * norm(b))

def find_best_match(face_embedding, threshold=0.40):
    similarities = [cosine_similarity(face_embedding, known_face) for known_face in known_faces]
    best_match_idx = np.argmax(similarities)
    best_score = similarities[best_match_idx]

    if best_score > threshold:
        return known_names[best_match_idx], best_score
    else:
        return "Unknown", best_score


In [13]:
os.getcwd()

'/mnt/f/keltron/projects/face_recognition/ver1'

In [14]:
# =============================
# 4. Inline Display Helper (for Jupyter)
# =============================
def show_frame_inline(frame):
    _, imarr = cv2.imencode('.jpg', frame)
    display(Image(data=imarr))
    clear_output(wait=True)

In [15]:
# =============================
# 5. Video Input
# =============================
vid_path = os.path.join("data_check", "video_of_christ_patt.mp4")
if not os.path.exists(vid_path):
    raise FileNotFoundError(f"❌ Video file not found at {vid_path}")

cap = cv2.VideoCapture(vid_path)
if not cap.isOpened():
    raise Exception("Cannot open video source. Check your camera or file path.")

print("\n✅ Face Recognition System Initialized. Press stop/interrupt to quit.\n")



✅ Face Recognition System Initialized. Press stop/interrupt to quit.



In [16]:

# =============================
# 6. Process Video Feed
# =============================
try:
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        faces = app.get(frame)

        for face in faces:
            bbox = face.bbox.astype(int)
            embedding = face.embedding
            name, score = find_best_match(embedding)

            if name != "Unknown":
                color = (0, 255, 0)
                shape = "◉"  # green circle marker for known
                label = f"{name} ({score:.2f})"
            else:
                color = (0, 0, 255)
                shape = "□"  # red box for unknown
                label = f"Unknown ({score:.2f})"

            # Draw bounding box and label
            cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
            cv2.putText(frame, label, (bbox[0], bbox[1] - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Show video inline (Jupyter safe)
        show_frame_inline(frame)

except KeyboardInterrupt:
    print("🛑 Interrupted by user. Closing stream gracefully...")
finally:
    cap.release()
    cv2.destroyAllWindows()

print("\n✅ Execution complete. System stable for Jupyter + WSL2 + Conda environment.")

error: OpenCV(4.12.0) /home/conda/feedstock_root/build_artifacts/libopencv_1755656420719/work/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [19]:
try :
    print(1/0)
except ZeroDivisionError as e:
    print(e)

division by zero


In [ ]:
# from time import sleep
# import matplotlib.pyplot as plt
# # =============================
# # 2. Setup video source
# # =============================
# video_source = vid_path  # Replace with your video file or use 0 for webcam
# cap = cv2.VideoCapture(video_source)

# if not cap.isOpened():
#     raise Exception("Cannot open video source. Check your camera or file path.")

# print("\n✅ Face Recognition System Initialized. Press 'Ctrl + C' to stop.\n")

# # =============================
# # 3. Dummy recognition placeholders
# # (Replace with your real InsightFace logic)
# # =============================
# def find_best_match(embedding):
#     # Example placeholder — replace with your actual recognition function
#     return "Person", 0.95


# class DummyFace:
#     def __init__(self, bbox, embedding):
#         self.bbox = bbox
#         self.embedding = embedding


# def dummy_get_faces(frame):
#     # Simulates detection (replace with your app.get(frame))
#     h, w, _ = frame.shape
#     return [DummyFace(bbox=[w//4, h//4, w//2, h//2], embedding=[0.1, 0.2, 0.3])]


# # =============================
# # 4. Frame processing loop
# # =============================
# frame_count = 0

# try:
#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             print("End of video or cannot read frame.")
#             break

#         faces = dummy_get_faces(frame)
#         for face in faces:
#             bbox = [int(x) for x in face.bbox]
#             embedding = face.embedding
#             name, score = find_best_match(embedding)

#             color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
#             cv2.rectangle(frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), color, 2)
#             cv2.putText(frame, f"{name} ({score:.2f})", (bbox[0], bbox[1] - 10),
#                         cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

#         # =============================
#         # Display frame using Matplotlib
#         # =============================
#         plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
#         plt.axis('off')
#         plt.title(f"Frame {frame_count}")
#         plt.show(block=False)
#         plt.pause(0.001)  # Allows Matplotlib to refresh display
#         plt.clf()         # Clears figure for next frame

#         frame_count += 1

# except KeyboardInterrupt:
#     print("\n🛑 Stopped by user.")

# finally:
#     cap.release()
#     plt.close()
#     print("\n✅ Execution complete. System stable for Jupyter/WSL2 environments.")
